# 실전 문제 해결 (부족한 데이터셋)

In [8]:
!pip3 install imblearn

You should consider upgrading via the 'pip install --upgrade pip' command.


In [11]:
import tensorflow as tf
import numpy as np
import random
from imblearn.over_sampling import BorderlineSMOTE

## 하이퍼 파라미터

In [6]:
EPOCHS = 100

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## 데이터셋 준비 (불균형한 데이터셋)

In [4]:
cifar10 = tf.keras.datasets.cifar10  # 32 x 32 x 3

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
 
# TODO: 학습 데이터를 Imbalanced small dataset으로 변형하기
x_train_small = list()
y_train_small = list()

for x, y in zip(x_train, y_train):
    if (y==0 and random.randint(0,100) < 10) or y == 1:
        x_train_small.append(x[:])  # 미리 flatten 해서 넣어줌
        y_train_small.append(y)
        
x_test_small = list()
y_test_small = list()

for x, y in zip(x_test, y_test):
    if y==0 or y == 1:
        x_test_small.append(x[:])  # 미리 flatten 해서 넣어줌
        y_test_small.append(y)

x_train = np.stack(x_train_small, axis=0)
y_train = np.stack(y_train_small, axis=0)

x_test = np.stack(x_test_small, axis=0)
y_test = np.stack(y_test_small, axis=0)


train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

## Keras API 모델 학습 (불균형한 데이터셋)

In [7]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                      tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

W0204 10:27:15.889901 140234464978688 base_layer.py:1814] Layer my_model_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Epoch 1/100
172/172 [==============================] - 3s 19ms/step - loss: 0.4251 - accuracy: 0.9034 - precision: 0.9275 - recall: 0.9700 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
172/172 [==============================] - 2s 13ms/step - loss: 0.2500 - accuracy: 0.9146 - precision: 0.9346 - recall: 0.9746 - val_loss: 0.5970 - val_accuracy: 0.7190 - val_precision: 0.6466 - val_recall: 0.9660
Epoch 3/100
172/172 [==============================] - 2s 13ms/step - loss: 0.2190 - accuracy: 0.9220 - precision: 0.9386 - recall: 0.9786 - val_loss: 1.0068 - val_accuracy: 0.6030 - val_precision: 0.5581 - val_recall: 0.9890
Epoch 4/100
172/172 [==============================] - 2s 13ms/step - loss: 0.2098 - accuracy: 0.9271 - precision: 0.9421 - recall: 0.9804 - val_loss: 0.6952 - val_accuracy: 0.7045 - val_precision: 0.6325 - val_recall: 0.9760
Epoch 5/100
172/172 [==============================] - 2s 13ms/step - loss: 0.18

172/172 [==============================] - 3s 15ms/step - loss: 0.0786 - accuracy: 0.9691 - precision: 0.9771 - recall: 0.9894 - val_loss: 0.7309 - val_accuracy: 0.7950 - val_precision: 0.7156 - val_recall: 0.9790
Epoch 38/100
172/172 [==============================] - 2s 15ms/step - loss: 0.0743 - accuracy: 0.9712 - precision: 0.9796 - recall: 0.9890 - val_loss: 0.6228 - val_accuracy: 0.8190 - val_precision: 0.7450 - val_recall: 0.9700
Epoch 39/100
172/172 [==============================] - 2s 14ms/step - loss: 0.0707 - accuracy: 0.9715 - precision: 0.9790 - recall: 0.9900 - val_loss: 0.4348 - val_accuracy: 0.8605 - val_precision: 0.8347 - val_recall: 0.8990
Epoch 40/100
172/172 [==============================] - 3s 15ms/step - loss: 0.0835 - accuracy: 0.9651 - precision: 0.9776 - recall: 0.9844 - val_loss: 0.6516 - val_accuracy: 0.8060 - val_precision: 0.7290 - val_recall: 0.9740
Epoch 41/100
172/172 [==============================] - 3s 15ms/step - loss: 0.0744 - accuracy: 0.9706 - 

172/172 [==============================] - 2s 14ms/step - loss: 0.0304 - accuracy: 0.9890 - precision: 0.9926 - recall: 0.9954 - val_loss: 0.9294 - val_accuracy: 0.8175 - val_precision: 0.7429 - val_recall: 0.9710
Epoch 74/100
172/172 [==============================] - 3s 15ms/step - loss: 0.0264 - accuracy: 0.9887 - precision: 0.9928 - recall: 0.9948 - val_loss: 0.9796 - val_accuracy: 0.8185 - val_precision: 0.7463 - val_recall: 0.9650
Epoch 75/100
172/172 [==============================] - 3s 15ms/step - loss: 0.0308 - accuracy: 0.9896 - precision: 0.9930 - recall: 0.9956 - val_loss: 1.0704 - val_accuracy: 0.8085 - val_precision: 0.7318 - val_recall: 0.9740
Epoch 76/100
172/172 [==============================] - 2s 15ms/step - loss: 0.0152 - accuracy: 0.9963 - precision: 0.9978 - recall: 0.9982 - val_loss: 1.3276 - val_accuracy: 0.7755 - val_precision: 0.6942 - val_recall: 0.9850
Epoch 77/100
172/172 [==============================] - 2s 14ms/step - loss: 0.0169 - accuracy: 0.9954 - 

## 데이터셋 준비 (BorderlineSMOTE)

In [13]:
# TODO: BorderlineSMOTE 적용하기

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1] * x_train.shape[2] * x_train.shape[3])).astype(np.float32)
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1] * x_test.shape[2] * x_test.shape[3])).astype(np.float32)

smote = BorderlineSMOTE()
x_train, y_train = smote.fit_resample(x_train, y_train)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Keras API 모델 학습 (BorderlineSMOTE)

In [14]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                      tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/100
313/313 [==============================] - 5s 16ms/step - loss: 0.5231 - accuracy: 0.7690 - precision: 0.7892 - recall: 0.7340 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
313/313 [==============================] - 4s 14ms/step - loss: 0.3343 - accuracy: 0.8602 - precision: 0.8917 - recall: 0.8200 - val_loss: 0.4491 - val_accuracy: 0.8030 - val_precision: 0.8507 - val_recall: 0.7350
Epoch 3/100
313/313 [==============================] - 4s 14ms/step - loss: 0.2678 - accuracy: 0.8972 - precision: 0.9247 - recall: 0.8648 - val_loss: 0.4486 - val_accuracy: 0.8150 - val_precision: 0.8786 - val_recall: 0.7310
Epoch 4/100
313/313 [==============================] - 4s 14ms/step - loss: 0.2305 - accuracy: 0.9141 - precision: 0.9414 - recall: 0.8832 - val_loss: 0.4187 - val_accuracy: 0.8370 - val_precision: 0.8474 - val_recall: 0.8220
Epoch 5/100
313/313 [==============================] - 4s 14ms/step - loss: 0.18

313/313 [==============================] - 4s 12ms/step - loss: 0.0352 - accuracy: 0.9896 - precision: 0.9930 - recall: 0.9862 - val_loss: 1.3111 - val_accuracy: 0.8065 - val_precision: 0.7299 - val_recall: 0.9730
Epoch 38/100
313/313 [==============================] - 4s 12ms/step - loss: 0.0322 - accuracy: 0.9889 - precision: 0.9912 - recall: 0.9866 - val_loss: 1.0109 - val_accuracy: 0.8495 - val_precision: 0.7900 - val_recall: 0.9520
Epoch 39/100
313/313 [==============================] - 4s 12ms/step - loss: 0.0188 - accuracy: 0.9945 - precision: 0.9966 - recall: 0.9924 - val_loss: 1.1429 - val_accuracy: 0.8365 - val_precision: 0.7656 - val_recall: 0.9700
Epoch 40/100
313/313 [==============================] - 4s 12ms/step - loss: 0.0287 - accuracy: 0.9903 - precision: 0.9934 - recall: 0.9872 - val_loss: 1.2180 - val_accuracy: 0.8130 - val_precision: 0.7378 - val_recall: 0.9710
Epoch 41/100
313/313 [==============================] - 4s 13ms/step - loss: 0.0212 - accuracy: 0.9935 - 

313/313 [==============================] - 4s 14ms/step - loss: 0.0047 - accuracy: 0.9988 - precision: 0.9990 - recall: 0.9986 - val_loss: 1.4015 - val_accuracy: 0.8395 - val_precision: 0.7758 - val_recall: 0.9550
Epoch 74/100
313/313 [==============================] - 4s 13ms/step - loss: 0.0060 - accuracy: 0.9988 - precision: 0.9994 - recall: 0.9982 - val_loss: 2.0709 - val_accuracy: 0.7830 - val_precision: 0.7036 - val_recall: 0.9780
Epoch 75/100
313/313 [==============================] - 4s 14ms/step - loss: 0.0062 - accuracy: 0.9980 - precision: 0.9982 - recall: 0.9978 - val_loss: 1.3222 - val_accuracy: 0.8460 - val_precision: 0.7845 - val_recall: 0.9540
Epoch 76/100
313/313 [==============================] - 4s 13ms/step - loss: 0.0086 - accuracy: 0.9971 - precision: 0.9976 - recall: 0.9966 - val_loss: 1.4087 - val_accuracy: 0.8325 - val_precision: 0.7671 - val_recall: 0.9550
Epoch 77/100
313/313 [==============================] - 4s 13ms/step - loss: 0.0477 - accuracy: 0.9838 - 